# Importation des données de Crimes.xlsx vers SQLite
Ce notebook charge un fichier Excel et insère ses données dans une base SQLite.

In [16]:

import sqlite3
import pandas as pd
import openpyxl

### Lecture du fichier Excel

In [20]:
file_path = r"C:\Users\JadS\Desktop\IUT\SAE_MigrationNoSQL\CSV_Modifies\Crimes.xlsx"
xls = pd.ExcelFile(file_path)

### Connexion à la base SQLite et création des tables

In [ ]:
# Connexion à la base SQLite
db_path = r"C:\Users\JadS\Desktop\IUT\SAE_MigrationNoSQL\BD\bdd"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Création des tables
cursor.executescript('''
CREATE TABLE IF NOT EXISTS Crime (
   code_index VARCHAR(50) PRIMARY KEY,
   lib_index VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Departement (
   id_dep VARCHAR(50) PRIMARY KEY,
   lib_dep VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Brigade (
   id_brigade VARCHAR(50) PRIMARY KEY,
   lib_brigade VARCHAR(50),
   id_dep VARCHAR(50),
   is_pn VARCHAR(50),
   id_dep_1 VARCHAR(50) NOT NULL,
   FOREIGN KEY(id_dep_1) REFERENCES Departement(id_dep)
);

CREATE TABLE IF NOT EXISTS PN (
   id_pn VARCHAR(50) PRIMARY KEY,
   id_dep VARCHAR(50),
   perimetre VARCHAR(50),
   id_brigade VARCHAR(50) NOT NULL UNIQUE,
   FOREIGN KEY(id_brigade) REFERENCES Brigade(id_brigade)
);

CREATE TABLE IF NOT EXISTS GN (
   id_gn VARCHAR(50) PRIMARY KEY,
   id_dep VARCHAR(50),
   id_brigade VARCHAR(50) NOT NULL UNIQUE,
   FOREIGN KEY(id_brigade) REFERENCES Brigade(id_brigade)
);

CREATE TABLE IF NOT EXISTS Enregistrer (
   code_index VARCHAR(50),
   id_brigade_1 VARCHAR(50),
   id_brigade VARCHAR(50),
   id_crime VARCHAR(50),
   annee VARCHAR(50),
   PRIMARY KEY(code_index, id_brigade_1),
   FOREIGN KEY(code_index) REFERENCES Crime(code_index),
   FOREIGN KEY(id_brigade_1) REFERENCES Brigade(id_brigade)
);
''')



In [19]:
# Insertion des données
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    df = df.iloc[2:].reset_index(drop=True)  # Suppression des en-têtes inutiles

    # Extraire les noms des CSP (PN) ou CGD (GN) à partir de la ligne 2, à droite de C2
    csp_cgd_list = df.columns[2:].unique().tolist()
    
    for csp_cgd in csp_cgd_list:
        cursor.execute("INSERT OR IGNORE INTO Brigade (lib_brigade, is_pn) VALUES (?, ?)",
                       (csp_cgd, "Services PN" in sheet_name))
        cursor.execute("SELECT id_brigade FROM Brigade WHERE lib_brigade = ?", (csp_cgd,))
        id_brigade = cursor.fetchone()
        
        if id_brigade:
            id_brigade = id_brigade[0]
            if isinstance(id_brigade, int):
                if "Services PN" in sheet_name:
                    cursor.execute("INSERT OR IGNORE INTO PN (lib_pn, perimetre, id_brigade) VALUES (?, ?, ?)",
                                   (csp_cgd, None, id_brigade))
                elif "Services GN" in sheet_name:
                    cursor.execute("INSERT OR IGNORE INTO GN (lib_gn, id_brigade) VALUES (?, ?)", (csp_cgd, id_brigade))
            else:
                print(f"Erreur: id_brigade non valide pour {csp_cgd}")

# Valider et fermer la connexion
conn.commit()
conn.close()

print("Base de données mise à jour avec succès !")

Base de données mise à jour avec succès !
